In [10]:
# 4. Retriever Runs: Fixing run_type and Document format

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Document", # FIX: Corrected missing key
          "metadata": {"source": "local_git_cache_v3"} # TWEAK: Custom metadata
      }
      for r in results
  ]

@traceable(
    run_type="retriever" # FIX: Set the correct run_type
)
def retrieve_docs(query):
  # TWEAK: Customized document content to be about Git fix strategy
  contents = ["The solution required deleting the local .git folder.", "A Personal Access Token (PAT) bypasses credential caching.", "Re-initialize Git and force push to establish tracking."] 
  return _convert_docs(contents)

print("\n--- Retriever Run Output ---")
print(retrieve_docs("How to fix Git Push errors?")) # TWEAK: Customized user query


--- Retriever Run Output ---
[{'page_content': 'The solution required deleting the local .git folder.', 'type': 'Document', 'metadata': {'source': 'local_git_cache_v3'}}, {'page_content': 'A Personal Access Token (PAT) bypasses credential caching.', 'type': 'Document', 'metadata': {'source': 'local_git_cache_v3'}}, {'page_content': 'Re-initialize Git and force push to establish tracking.', 'type': 'Document', 'metadata': {'source': 'local_git_cache_v3'}}]


In [9]:
# 3. Streaming LLM Runs: Fixing the reduce_fn

def _reduce_chunks(chunks: list):
    # This reduction logic is for LangSmith, ensuring the streamed response is combined
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    # The output format is now clean and avoids syntax issues:
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "my_provider", "ls_model_name": "my_model"},
    reduce_fn=_reduce_chunks # FIX: Added the required reduce_fn
)
def my_streaming_chat_model(messages: list):
    # TWEAK: Customized chunk output based on the input
    user_input = messages[1]["content"]
    for chunk in [f"Hello, {user_input} is a great name.", " I hope you have a nice day!"]:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

print("\n--- Streaming Model Run Output ---")
list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "You are a helpful assistant. Please greet the user."},
            {"role": "user", "content": "Ranger the Retriever"}, # TWEAK: Changed animal input
        ],
    )
)


--- Streaming Model Run Output ---


[{'choices': [{'message': {'content': 'Hello, Ranger the Retriever is a great name.',
     'role': 'assistant'}}]},
 {'choices': [{'message': {'content': ' I hope you have a nice day!',
     'role': 'assistant'}}]}]

In [8]:
# 2. LLM Runs: Setting run_type="llm" and adding custom example

inputs = [
  {"role": "system", "content": "You are a friendly barista assistant. Please confirm the customer's coffee order."},
  {"role": "user", "content": "I'd like a large cold brew with vanilla syrup."}, # TWEAK: Modified input for a coffee order
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "One large cold brew with vanilla added to your order. Will that be all?", # TWEAK: Modified output
          }
      }
  ]
}

@traceable(
    run_type="llm",  # FIX: Set the correct run_type
    metadata={
        "ls_provider": "synthetic-groq-test", 
        "ls_model_name": "barista-bot-v1" # TWEAK: Custom model name
    } 
)
def chat_model(messages: list):
  # In a real app, this would be an actual Groq API call, but for the lesson,
  # we return the expected output format for tracing.
  return output

print("--- Chat Model Run Output ---")
# To fully trace this LLM run, we pass it to LangChain's invocation method.
from langchain.schema.messages import HumanMessage
response = llm_client.invoke([HumanMessage(content="Simulate call to chat_model")])
print(response.content)

--- Chat Model Run Output ---
I'll simulate a call to a chat model. To start, I'll need to know what kind of input or prompt you'd like to give the model. Would you like to:

1. Ask a question
2. Provide a statement to discuss
3. Engage in a role-playing scenario
4. Something else (please specify)

Please respond with the number of your chosen action or describe your desired interaction.


In [7]:
# 1. Setup Environment and Imports (Groq & Generic)
import os
from dotenv import load_dotenv
from langsmith import traceable 
from typing import List
import nest_asyncio
import json
# Use Groq LLM client
from langchain_groq import ChatGroq

# --- Configuration ---
load_dotenv()
MODEL_NAME = "llama-3.3-70b-versatile" # Using a supported Groq model
llm_client = ChatGroq(model=MODEL_NAME)
nest_asyncio.apply()